In [1]:
import numpy as np 
import pandas as pd
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

print(os.listdir("../input"))

Using TensorFlow backend.


['Hindi_English_Truncated_Corpus.csv']


In [2]:
df=pd.read_csv("../input/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')

In [3]:
df['source'].value_counts()

tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64

In [4]:
df=df[df['source']=='ted']

In [5]:
df.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है


In [6]:
pd.isnull(df).sum()

source              0
english_sentence    0
hindi_sentence      0
dtype: int64

In [7]:
df=df[~pd.isnull(df['english_sentence'])]

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df=df.sample(n=25000,random_state=42)
df.shape

(25000, 3)

In [10]:
# Lowercase all characters
df['english_sentence']=df['english_sentence'].apply(lambda x: x.lower())
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: x.lower())

In [11]:
# Remove quotes
df['english_sentence']=df['english_sentence'].apply(lambda x: re.sub("'", '', x))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [12]:
# Remove all the special characters
exclude = set(string.punctuation) 
df['english_sentence']=df['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [13]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
df['english_sentence']=df['english_sentence'].apply(lambda x: x.translate(remove_digits))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
df['english_sentence']=df['english_sentence'].apply(lambda x: x.strip())
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: x.strip())
df['english_sentence']=df['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))


In [14]:
# Add start and end tokens to target sequences
df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [15]:
df.head()

,source,english_sentence,hindi_sentence
82040,ted,we still dont know who her parents are who she is,START_ हम अभी तक नहीं जानते हैं कि उसके मातापि...
85038,ted,no keyboard,START_ कोई कुंजीपटल नहीं _END
58018,ted,but as far as being a performer,START_ लेकिन एक कलाकार होने के साथ _END
74470,ted,and this particular balloon,START_ और यह खास गुब्बारा _END
122330,ted,and its not as hard as you think integrate cli...,START_ और जितना आपको लगता है यह उतना कठिन नहीं...


In [16]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in df['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in df['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [17]:
len(all_eng_words)

14030

In [18]:
len(all_hindi_words)

17540

In [19]:
df['length_eng_sentence']=df['english_sentence'].apply(lambda x:len(x.split(" ")))
df['length_hin_sentence']=df['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [20]:
df.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
82040,ted,we still dont know who her parents are who she is,START_ हम अभी तक नहीं जानते हैं कि उसके मातापि...,11,16
85038,ted,no keyboard,START_ कोई कुंजीपटल नहीं _END,2,5
58018,ted,but as far as being a performer,START_ लेकिन एक कलाकार होने के साथ _END,7,8
74470,ted,and this particular balloon,START_ और यह खास गुब्बारा _END,4,6
122330,ted,and its not as hard as you think integrate cli...,START_ और जितना आपको लगता है यह उतना कठिन नहीं...,16,20


In [21]:
df[df['length_eng_sentence']>30].shape

(0, 5)

In [22]:
df=df[df['length_eng_sentence']<=20]
df=df[df['length_hin_sentence']<=20]

In [23]:
df.shape

(24774, 5)

In [24]:
print("maximum length of Hindi Sentence ",max(df['length_hin_sentence']))
print("maximum length of English Sentence ",max(df['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [25]:
max_length_src=max(df['length_hin_sentence'])
max_length_tar=max(df['length_eng_sentence'])

In [26]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(14030, 17540)

In [27]:
num_decoder_tokens += 1 

In [28]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [29]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [30]:
df = shuffle(df)
df.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
50095,ted,crowd noise so in a cocktail party like this,START_ भीड़ का शोर तो इस तरह की कॉकटेल पार्टी ...,9,12
100596,ted,there are about cases of polio,START_ तो पोलिओ के करीब उदाहरण _END,6,7
14182,ted,or “how can we help”,START_ या फिर “हम कैसे मदद कर सकते हैं” _END,5,10
9868,ted,not able to provide them with vitamin a,START_ उन्हें विटामिन ए न दे कर। _END,8,8
100561,ted,“and what do you mean with western world” i said,START_ “और पश्चिमी दुनिया से आपका क्या तात्पर्...,10,12


In [31]:
X, y = df['english_sentence'], df['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((19819,), (4955,))

In [32]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    #encode the input sequence
                    encoder_input_data[i, t] = input_token_index[word] 
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        #decode the input sequence
                        decoder_input_data[i, t] = target_token_index[word] 
                    if t>0:
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [33]:
# Encoder
latent_dim=300
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [34]:
# Set up the decoder, using encoder_states as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [35]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [36]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    4209000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    5262300     input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [56]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

In [57]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/100
154/154 [==============================] - 62s 400ms/step - loss: 0.7513 - val_loss: 6.7649
Epoch 2/100
154/154 [==============================] - 62s 401ms/step - loss: 0.6915 - val_loss: 6.8033
Epoch 3/100
154/154 [==============================] - 62s 401ms/step - loss: 0.6601 - val_loss: 6.8397
Epoch 4/100
154/154 [==============================] - 62s 402ms/step - loss: 0.6282 - val_loss: 6.8613
Epoch 5/100
154/154 [==============================] - 62s 402ms/step - loss: 0.6024 - val_loss: 6.8836
Epoch 6/100
154/154 [==============================] - 62s 404ms/step - loss: 0.5772 - val_loss: 6.9161
Epoch 7/100
154/154 [==============================] - 63s 407ms/step - loss: 0.5547 - val_loss: 6.9458
Epoch 8/100
154/154 [==============================] - 63s 411ms/step - loss: 0.5349 - val_loss: 6.9721
Epoch 9/100
154/154 [==============================] - 62s 404ms/step - loss: 0.5110 - val_loss: 7.0080
Epoch 10/100
154/154 [==============================] - 62s 399m

In [58]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
dec_emb2= dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) 
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [59]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_token_index['START_']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]
    return decoded_sentence

In [60]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [61]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: and then millions
Actual Hindi Translation:  और अधिक जोश में आ गया 
Predicted Hindi Translation:  और अधिक जोश में आ गया 


In [62]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: such as cooking or making ice sculptures
Actual Hindi Translation:  जैसे कि खाना बनाना या बर्फ की प्रतिमा बनाना 
Predicted Hindi Translation:  जैसे कि खाना बनाना या बर्फ की प्रतिमा बनाना 


In [63]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: i was there to report on that story
Actual Hindi Translation:  मैं वहाँ उस कहानी पर रिपोर्ट करने गया था 
Predicted Hindi Translation:  मैं वहाँ उस कहानी पर रिपोर्ट करने गया था 


In [64]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: because the transformation to amateur media
Actual Hindi Translation:  क्योंकि शौकिया मीडिया के लिए परिवर्तन 
Predicted Hindi Translation:  क्योंकि शौकिया मीडिया के लिए परिवर्तन 
